# Eseguo la pulizia di base
## dal 2004 al 2019
---
Se mancava la classe metto -1

In [3]:
import numpy as np
import pandas as pd

In [4]:
def classe(str):
    if str.startswith("I "):
        return 1
    elif str.startswith("II "):
        return 2
    elif str.startswith("III "):
        return 3
    elif str.startswith("IV "):
        return 4
    elif str.startswith("V "):
        return 5
    else:
        return -1

def medaglia(str):
    if (str == "oro") | (str == "Oro"):
        return "ORO"
    elif (str == "argento") | (str == "Argento"):
        return "ARGENTO"
    elif (str == "bronzo") | (str == "Bronzo"):
        return "BRONZO"
    elif (str == "menzione") | (str == "Menzione"):
        return "MENZIONE"
    else:
        return "-"

df_vuoto = pd.DataFrame(columns=["anno", "studente", "posizione", "regione", "classe", "es1", "es2", "es3", "es4", "es5", "es6", "tot", "medaglia"])
df_vuoto

,anno,studente,posizione,regione,classe,es1,es2,es3,es4,es5,es6,tot,medaglia


In [5]:
def pulisci_df(anno, xx, numPartecipanti):
    
    # Prendo i due dataframe
    df = pd.read_csv("data/datiSporchi/cese"+xx+".csv", encoding="utf8")
    cese = df_vuoto

    # Inizio a popolare quello bello
    for i in range(numPartecipanti):
        cese.at[i, "anno"] = anno
    
        cese.at[i, "studente"] = df.at[i*3, "studente"]
        # La posizione si sistema più avanti
        cese.at[i, "regione"] = df.at[i*3+1, "distretto"]
        cese.at[i, "classe"] = classe( df.at[i*3+2, "studente"] )
    
        cese.at[i, "es1"] = int( df.at[i*3, "es 1"] )
        cese.at[i, "es2"] = int( df.at[i*3, "es 2"] )
        cese.at[i, "es3"] = int( df.at[i*3, "es 3"] )
        cese.at[i, "es4"] = int( df.at[i*3, "es 4"] )
        cese.at[i, "es5"] = int( df.at[i*3, "es 5"] )
        cese.at[i, "es6"] = int( df.at[i*3, "es 6"] )

        cese.at[i, "tot"] = int( df.at[i*3, "tot"] )
        cese.at[i, "medaglia"] = medaglia( df.at[i*3, "medaglia"] )

    # Cambio il tipo dei dati
    cese = cese.astype({"anno": int, "studente": str, "posizione": float, "regione": str, "classe": int,
                        "es1": int, "es2": int, "es3": int, "es4": int, "es5": int, "es6": int,
                        "tot": int, "medaglia": str})

    # Ordino i dati per punteggio
    cese = cese.sort_values("tot", ascending=False)

    # Sistemo l'indice
    cese = cese.reset_index(drop=True)

    # Popolo la colonna della posizione in classifica
    pos = 1
    cese.at[0, "posizione"] = pos
    for i in range(1, len(cese.index)):
        if cese.at[i, "tot"] == cese.at[i-1, "tot"]:
            #cese.at[i, "posizione"] = cese.at[i-1, "posizione"]
            cese.at[i, "posizione"] = pos
        else:
            cese.at[i, "posizione"] = i+1
            pos = i+1

    cese = cese.astype({"posizione": int})

    # Assegno le menzioni se mancanti
    cese = assegna_menzioni(cese)

    # Salvo il database su pickle
    cese.to_pickle("data/datiBasePuliti/cese_"+xx+".pickle")

In [6]:
def assegna_menzioni(df):
    mask_1 = df["es1"] == 7
    mask_2 = df["es2"] == 7
    mask_3 = df["es3"] == 7
    mask_4 = df["es4"] == 7
    mask_5 = df["es5"] == 7
    mask_6 = df["es6"] == 7
    mask_a = df["medaglia"] == "-"
    mask_papabile = mask_1 | mask_2 | mask_3 | mask_4 | mask_5 | mask_6
    mask_tot = mask_papabile & mask_a

    i = 0
    for b in mask_tot:
        if b:
            df.at[i, "medaglia"] = "MENZIONE"
        i = i+1

    return df

## 2004

In [7]:
pulisci_df(2004, "04", 300)

In [8]:
recupero_04 = pd.read_pickle("data/datiBasePuliti/cese_04.pickle")
recupero_04[ recupero_04["medaglia"] == "MENZIONE" ]

,anno,studente,posizione,regione,classe,es1,es2,es3,es4,es5,es6,tot,medaglia
130,2004,Sirico Domenico,131,(Campania),4,7,3,0,0,0,0,10,MENZIONE
131,2004,Petriaggi Massimo,131,(Umbria),4,7,3,0,0,0,0,10,MENZIONE
132,2004,Riello Aldo,131,(Veneto),3,7,3,0,0,0,0,10,MENZIONE
133,2004,Sala Giovanni,131,(Lombardia),5,7,3,0,0,0,0,10,MENZIONE
134,2004,Servasi Michele,131,(Marche),5,7,0,0,3,0,0,10,MENZIONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,2004,Frigerio Matteo,202,(Lombardia),4,7,0,0,0,0,0,7,MENZIONE
235,2004,Gargiulo Fernando,202,(Campania),5,7,0,0,0,0,0,7,MENZIONE
236,2004,Gliori Henry,202,(Toscana),5,7,0,0,0,0,0,7,MENZIONE
237,2004,Grande Francesco,202,(Piemonte),3,7,0,0,0,0,0,7,MENZIONE


In [9]:
recupero_04

,anno,studente,posizione,regione,classe,es1,es2,es3,es4,es5,es6,tot,medaglia
0,2004,Barbieri Luca,1,(Lombardia),4,7,4,7,6,7,7,38,ORO
1,2004,Scala Riccardo,2,(Lazio),5,7,3,7,7,6,7,37,ORO
2,2004,Golla Marco,3,(Veneto),5,7,3,7,1,7,7,32,ORO
3,2004,Biasini Riccardo,4,(Emilia Romagna),5,7,3,7,7,7,0,31,ORO
4,2004,Canciani Andrea,4,(Liguria),4,7,4,0,7,6,7,31,ORO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2004,Lupini Martino,289,(Marche),4,0,0,0,0,0,0,0,-
296,2004,Giuliano Antonio,289,(Puglia),4,0,0,0,0,0,0,0,-
297,2004,Dallorto Fabio,289,(Piemonte),4,0,0,0,0,0,0,0,-
298,2004,Bertinelli Alessandro,289,(Emilia Romagna),1,0,0,0,0,0,0,0,-


# 2005

In [10]:
pulisci_df(2005, "05", 290)
rec_05 = pd.read_pickle("data/datiBasePuliti/cese_05.pickle")
rec_05

,anno,studente,posizione,regione,classe,es1,es2,es3,es4,es5,es6,tot,medaglia
0,2005,Negro Gabriele,1,(),-1,7,7,7,7,7,7,42,ORO
1,2005,Di Marino Simone,2,(),-1,7,7,7,5,7,7,40,ORO
2,2005,Cavazzani Francesco,2,(),-1,7,6,7,7,6,7,40,ORO
3,2005,Barbieri Luca,4,(),-1,7,7,7,7,5,6,39,ORO
4,2005,Attanasio Stefano,5,(),-1,7,7,7,7,1,7,36,ORO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2004,Manai Andrea,290,(Sardegna),4,0,0,0,0,0,0,0,-
296,2004,Lupini Martino,290,(Marche),4,0,0,0,0,0,0,0,-
297,2004,Di Mille Cosmo,290,(Lazio),1,0,0,0,0,0,0,0,-
298,2005,Lagana' Fabio,290,(),-1,0,0,0,0,0,0,0,-


# 2006

In [11]:
pulisci_df(2006, "06", 296)
rec_06 = pd.read_pickle("data/datiBasePuliti/cese_06.pickle")
rec_06

,anno,studente,posizione,regione,classe,es1,es2,es3,es4,es5,es6,tot,medaglia
0,2006,Di Marino Simone,1,(Abruzzo),5,7,7,6,7,7,6,40,ORO
1,2006,Cavazzani Francesco,2,(Lazio),5,7,7,7,7,5,5,38,ORO
2,2006,Colombo Maria,3,(Lombardia),3,7,7,7,7,7,1,36,ORO
3,2006,Bianchi Claudio Filippo,4,(Lombardia),5,7,7,6,7,5,0,32,ORO
4,2006,Santi Elia,4,(Emilia Romagna),4,7,5,6,7,7,0,32,ORO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2006,Buccheri Giuseppe,274,(Sicilia),5,0,0,0,0,0,0,0,-
296,2006,Bonsignore Fulvio,274,(Sicilia),4,0,0,0,0,0,0,0,-
297,2006,And� Alessia,274,(Sicilia),2,0,0,0,0,0,0,0,-
298,2006,Amoroso Luca,274,(Sicilia),3,0,0,0,0,0,0,0,-


# 2007

In [12]:
pulisci_df(2007, "07", 300)
rec_07 = pd.read_pickle("data/datiBasePuliti/cese_07.pickle")
rec_07

,anno,studente,posizione,regione,classe,es1,es2,es3,es4,es5,es6,tot,medaglia
0,2007,Vertechi Pietro,1,(Lazio),3,7,7,7,7,7,7,42,ORO
1,2007,Fogari Andrea,2,(Friuli Venezia Giulia),2,7,7,7,7,7,4,39,ORO
2,2007,Colombo Maria,2,(Lombardia),4,7,7,7,7,7,4,39,ORO
3,2007,De Capua Antonio,4,(Campania),5,7,7,7,7,7,0,35,ORO
4,2007,Lombardo Davide,4,(Liguria),4,7,7,7,7,7,0,35,ORO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2007,Bassu Cesare,288,(Sardegna),4,0,0,0,0,0,0,0,-
296,2007,Aversa Bruno,288,(Campania),2,0,0,0,0,0,0,0,-
297,2007,Amoroso Gianluca,288,(Sicilia),3,0,0,0,0,0,0,0,-
298,2007,Altieri Silvestro,288,(Campania),4,0,0,0,0,0,0,0,-


# 2008 - 2019

In [13]:
pulisci_df(2008, "08", 300)
pulisci_df(2009, "09", 302)
pulisci_df(2010, "10", 300)

In [14]:
rec_09 = pd.read_pickle("data/datiBasePuliti/cese_09.pickle")
rec_09

,anno,studente,posizione,regione,classe,es1,es2,es3,es4,es5,es6,tot,medaglia
0,2009,Bianchi Andrea,1,(Lazio),3,7,7,7,7,7,7,42,ORO
1,2009,Ghidelli Luca,1,(Lombardia),4,7,7,7,7,7,7,42,ORO
2,2009,Fogari Andrea,3,(Friuli Venezia Giulia),4,7,7,7,7,7,4,39,ORO
3,2009,Paolini Giovanni,4,(Lombardia),4,7,7,7,7,7,3,38,ORO
4,2009,Kuzmin Kirill,5,(Piemonte),5,7,7,6,7,7,3,37,ORO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2009,Felicetti Riccardo,288,(Marche),4,0,0,0,0,0,0,0,-
298,2009,Famoso Francesco,288,(Campania),2,0,0,0,0,0,0,0,-
299,2009,Essolino Attilio,288,(Campania),3,0,0,0,0,0,0,0,-
300,2009,Alberigi Federico,288,(Toscana),2,0,0,0,0,0,0,0,-


In [15]:
pulisci_df(2011, "11", 300)
pulisci_df(2012, "12", 309)
pulisci_df(2013, "13", 308)
pulisci_df(2014, "14", 297)

In [16]:
pulisci_df(2015, "15", 302)
pulisci_df(2016, "16", 303)
pulisci_df(2017, "17", 301)
pulisci_df(2018, "18", 300)
pulisci_df(2019, "19", 300)

In [17]:
rec_17 = pd.read_pickle("data/datiBasePuliti/cese_17.pickle")
rec_17

,anno,studente,posizione,regione,classe,es1,es2,es3,es4,es5,es6,tot,medaglia
0,2017,Chen Jacopo Guoyi,1,(Lazio),5,7,7,7,7,7,7,42,ORO
1,2017,Viola Federico,2,(Lazio),4,7,7,7,7,7,4,39,ORO
2,2017,Ulliana Andrea,3,(Veneto),5,7,7,7,7,7,1,36,ORO
3,2017,Deniskin Nikita,4,(Campania),5,7,7,7,7,7,0,35,ORO
4,2017,Tarini Bernardo,4,(Toscana),3,7,5,7,2,7,7,35,ORO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,2013,Scollo Giuseppe,298,(Sicilia),3,0,0,0,0,0,0,0,-
305,2013,Spadini Riccardo,298,(Marche),4,0,0,0,0,0,0,0,-
306,2013,Tatti Antonio,298,(Sardegna),5,0,0,0,0,0,0,0,-
307,2013,Wang Lei,298,(Sicilia),-1,0,0,0,0,0,0,0,-


In [18]:
rec_19 = pd.read_pickle("data/datiBasePuliti/cese_19.pickle")
rec_19[ rec_19["tot"] == 3]

,anno,studente,posizione,regione,classe,es1,es2,es3,es4,es5,es6,tot,medaglia
240,2019,Lugli Francesco,241,(Emilia-Romagna),1,1,1,1,0,0,0,3,-
241,2019,Zoppi Giorgio,241,(Marche),5,0,2,1,0,0,0,3,-
242,2019,Vasi Salvatore,241,(Puglia),5,0,1,1,0,1,0,3,-
243,2019,Rizzotti Lorenzo,241,(Veneto),4,1,0,2,0,0,0,3,-
244,2019,Rigoli Matteo,241,(Lombardia),3,0,2,1,0,0,0,3,-
245,2019,Querqui Alessandro,241,(Lazio),3,2,0,1,0,0,0,3,-
246,2019,Piazza Alec,241,(Lombardia),4,0,2,1,0,0,0,3,-
247,2019,Mastrangelo Salvatore,241,(Molise),3,2,0,0,0,1,0,3,-
248,2019,Marinaro Gabriele,241,(Abruzzo),4,0,2,1,0,0,0,3,-
249,2019,Italiano Giacomo,241,(Puglia),1,0,0,1,0,1,1,3,-


In [19]:
rec_17[rec_17["anno"] != 2017]

,anno,studente,posizione,regione,classe,es1,es2,es3,es4,es5,es6,tot,medaglia
301,2016,Zamparelli Francesca,298,(Campania),2,0,0,0,0,0,0,0,-
302,2016,Zaninotto Alessio,298,(Trentino Alto Adige),4,0,0,0,0,0,0,0,-
303,2013,Resta Roberta,298,(Puglia),3,0,0,0,0,0,0,0,-
304,2013,Scollo Giuseppe,298,(Sicilia),3,0,0,0,0,0,0,0,-
305,2013,Spadini Riccardo,298,(Marche),4,0,0,0,0,0,0,0,-
306,2013,Tatti Antonio,298,(Sardegna),5,0,0,0,0,0,0,0,-
307,2013,Wang Lei,298,(Sicilia),-1,0,0,0,0,0,0,0,-
308,2012,Pugliese Vincenzo,298,(Calabria),5,0,0,0,0,0,0,0,-
